In [1]:
from typing import Iterator, List, Dict

In [2]:
import torch
import torch.optim as optim
import numpy as np

In [3]:
from allennlp.common.params import Params

In [4]:
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [43]:
from allennlp.data.dataset_readers import DatasetReader

In [6]:
from allennlp.common.file_utils import cached_path

In [7]:
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token

In [8]:
from allennlp.data.vocabulary import Vocabulary

In [9]:
from allennlp.models import Model

In [10]:
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits

In [11]:
from allennlp.training.metrics import CategoricalAccuracy

In [12]:
from allennlp.data.iterators import BucketIterator

In [13]:
from allennlp.training.trainer import Trainer

In [14]:
from allennlp.predictors import SentenceTaggerPredictor

torch.manual_seed(1)

In [34]:
from allennlp.data.token_indexers import TokenCharactersIndexer
import unicodedata, glob, os, string, random

In [83]:
# Methods from pytorch tutorial

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

def findFiles(path): return glob.glob(path)

class Dataset_Reader(DatasetReader):
    
    # Implementing data reader from character RNN tutorial
    def __init__(self, token_indexers: Dict[str, SingleIdTokenIndexer] = None) -> None:
        super().__init__(lazy=False)
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        # Character indexers based on AllenNLP tutorial
        self.token_character_indexers = {"token_characters" : TokenCharactersIndexer()}
        
    def text_to_instance(self, tokens: List[str], tags: List[str] = None) -> Instance:
        name_tokens = [Token(name) for name in tokens]
        name_token_field = TextField(name_tokens, self.token_indexers)
        # Same logic for character tokens and fields as well
        fields = {"tokens": name_token_field}
        name_token_character_field = TextField(name_tokens, self.token_character_indexers)
        fields["token_characters"] = name_token_character_field
        if tags:
            label_field = SequenceLabelField(labels=tags, sequence_field=name_token_field)
            fields["labels"] = label_field

        return Instance(fields)
    
    def _read(self, file_path: str) -> Iterator[Instance]:
        concatenated_pair = []
        for filename in findFiles(file_path):
            category = os.path.splitext(os.path.basename(filename))[0]
            all_categories.append(category)
            lines = readLines(filename)
            category_lines[category] = lines
            concatenated_pair.extend([(word, category) for word in lines])
        # Tried using names in bigger (language based) splits, did not work out, will be reported in the report
        # Make smaller splits someway
        # Trying making splits of 30 names. for that shuffle the names, otherwise same problem as before
        np.random.shuffle(concatenated_pair)
        #print(concatenated_pair)
        # Code for chunks from "https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks"
        for i in range(0, len(concatenated_pair), 30):
            shuffeled_split = concatenated_pair[i:i+30]
            yield self.text_to_instance([pair[0] for pair in shuffeled_split], [pair[1] for pair in shuffeled_split])

In [84]:
dataset_reader = Dataset_Reader()

/home/joonas/.conda/envs/allennlp/lib/python3.6/site-packages/allennlp/data/token_indexers/token_characters_indexer.py:51: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)


In [85]:
category_lines = {}
all_categories = []
names_data = dataset_reader.read('data/names/*.txt')


0it [00:00, ?it/s]
669it [00:00, 6581.72it/s]

In [86]:
class NamesClassifier(Model):
    def __init__(self,
                 word_embeddings: TextFieldEmbedder,
                 encoder: Seq2SeqEncoder,
                 vocab: Vocabulary) -> None:
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                          out_features=vocab.get_vocab_size('labels'))
        self.accuracy = CategoricalAccuracy()
    def forward(self,
                tokens: Dict[str, torch.Tensor],
                token_characters: Dict[str, torch.Tensor],
                labels: torch.Tensor = None) -> Dict[str, torch.Tensor]:
        mask = get_text_field_mask(tokens)
        # Had to change the input for embeddings, give it as an input of dictionary with tokens and token characters
        # use ** to unpack them
        embeddings = self.word_embeddings({**tokens, **token_characters})
        encoder_out = self.encoder(embeddings, mask)
        tag_logits = self.hidden2tag(encoder_out)
        output = {"tag_logits": tag_logits}
        if labels is not None:
            self.accuracy(tag_logits, labels, mask)
            output["loss"] = sequence_cross_entropy_with_logits(tag_logits, labels, mask)

        return output
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {"accuracy": self.accuracy.get_metric(reset)}

In [87]:
# Since we don't have different files for validation, use sklearn to split the data
from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(names_data, test_size=0.2)
vocab = Vocabulary.from_instances(names_data)


100%|██████████| 669/669 [00:00<00:00, 3611.33it/s]

In [88]:
vocab

Vocabulary with namespaces:  tokens, Size: 17423 || token_characters, Size: 57 || labels, Size: 18 || Non Padded Namespaces: {'*tags', '*labels'}

In [89]:
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
from allennlp.modules.token_embedders.token_characters_encoder import TokenCharactersEncoder

# Define the model
WORD_EMBEDDING_DIM = 3
CHAR_EMBEDDING_DIM = 3
HIDDEN_DIM = 6
EMBEDDING_DIM = WORD_EMBEDDING_DIM + CHAR_EMBEDDING_DIM

# Embeddings for characters and then for names
character_encoder = PytorchSeq2VecWrapper(torch.nn.RNN(CHAR_EMBEDDING_DIM, CHAR_EMBEDDING_DIM, batch_first=True))
token_character_embedding = Embedding(num_embeddings=vocab.get_vocab_size('token_characters'),
                            embedding_dim=WORD_EMBEDDING_DIM)
character_embeddings = TokenCharactersEncoder(token_character_embedding, character_encoder)

# Now names
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=WORD_EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding, "token_characters" : character_embeddings})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
model = NamesClassifier(word_embeddings, lstm, vocab)

In [90]:
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1

In [92]:
# Train the model - 30 epochs seem to give a pretty good baseline accuracy - 0.7 val accuracy
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens"), ("token_characters", "num_token_characters")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_set,
                  validation_dataset=val_set,
                  patience=10,
                  num_epochs=30, 
                  cuda_device=cuda_device)
trainer.train()


  0%|          | 0/268 [00:00<?, ?it/s]
accuracy: 0.5222, loss: 1.4904 ||:   1%|          | 3/268 [00:00<00:09, 27.78it/s]
accuracy: 0.5619, loss: 1.4109 ||:   3%|▎         | 7/268 [00:00<00:09, 28.75it/s]
accuracy: 0.5650, loss: 1.4230 ||:   4%|▎         | 10/268 [00:00<00:09, 27.75it/s]
accuracy: 0.5625, loss: 1.4189 ||:   4%|▍         | 12/268 [00:00<00:11, 21.37it/s]
accuracy: 0.5607, loss: 1.4317 ||:   5%|▌         | 14/268 [00:00<00:12, 19.61it/s]
accuracy: 0.5559, loss: 1.4497 ||:   6%|▋         | 17/268 [00:00<00:11, 21.68it/s]
accuracy: 0.5617, loss: 1.4420 ||:   7%|▋         | 20/268 [00:00<00:11, 22.40it/s]
accuracy: 0.5601, loss: 1.4440 ||:   9%|▊         | 23/268 [00:00<00:10, 22.86it/s]
accuracy: 0.5603, loss: 1.4412 ||:  10%|▉         | 26/268 [00:01<00:11, 21.33it/s]
accuracy: 0.5615, loss: 1.4375 ||:  11%|█         | 29/268 [00:01<00:11, 21.36it/s]
accuracy: 0.5603, loss: 1.4478 ||:  12%|█▏        | 32/268 [00:01<00:11, 20.70it/s]
accuracy: 0.5585, loss: 1.4492 ||:  1

accuracy: 0.5515, loss: 1.4453 ||:  16%|█▋        | 11/67 [00:00<00:01, 44.40it/s]
accuracy: 0.5433, loss: 1.4669 ||:  22%|██▏       | 15/67 [00:00<00:01, 41.57it/s]
accuracy: 0.5447, loss: 1.4495 ||:  33%|███▎      | 22/67 [00:00<00:00, 45.36it/s]
accuracy: 0.5417, loss: 1.4621 ||:  39%|███▉      | 26/67 [00:00<00:01, 40.84it/s]
accuracy: 0.5394, loss: 1.4651 ||:  45%|████▍     | 30/67 [00:00<00:00, 38.90it/s]
accuracy: 0.5412, loss: 1.4727 ||:  51%|█████     | 34/67 [00:00<00:00, 39.19it/s]
accuracy: 0.5448, loss: 1.4747 ||:  72%|███████▏  | 48/67 [00:00<00:00, 49.55it/s]
accuracy: 0.5481, loss: 1.4734 ||:  93%|█████████▎| 62/67 [00:01<00:00, 61.21it/s]
accuracy: 0.5478, loss: 1.4724 ||: 100%|██████████| 67/67 [00:01<00:00, 64.29it/s]
  0%|          | 0/268 [00:00<?, ?it/s]
accuracy: 0.5333, loss: 1.4901 ||:   1%|▏         | 4/268 [00:00<00:07, 33.89it/s]
accuracy: 0.5479, loss: 1.4573 ||:   3%|▎         | 8/268 [00:00<00:08, 30.98it/s]
accuracy: 0.5455, loss: 1.4435 ||:   4%|▍      

  0%|          | 0/67 [00:00<?, ?it/s]
accuracy: 0.5222, loss: 1.4559 ||:   4%|▍         | 3/67 [00:00<00:02, 28.73it/s]
accuracy: 0.5646, loss: 1.3812 ||:  12%|█▏        | 8/67 [00:00<00:01, 32.87it/s]
accuracy: 0.5482, loss: 1.4735 ||:  28%|██▊       | 19/67 [00:00<00:01, 40.76it/s]
accuracy: 0.5590, loss: 1.4449 ||:  36%|███▌      | 24/67 [00:00<00:01, 40.31it/s]
accuracy: 0.5609, loss: 1.4377 ||:  43%|████▎     | 29/67 [00:00<00:00, 41.80it/s]
accuracy: 0.5544, loss: 1.4422 ||:  51%|█████     | 34/67 [00:00<00:00, 41.88it/s]
accuracy: 0.5481, loss: 1.4736 ||:  64%|██████▍   | 43/67 [00:00<00:00, 48.99it/s]
accuracy: 0.5456, loss: 1.4798 ||:  73%|███████▎  | 49/67 [00:00<00:00, 48.87it/s]
accuracy: 0.5494, loss: 1.4724 ||:  82%|████████▏ | 55/67 [00:01<00:00, 49.82it/s]
accuracy: 0.5530, loss: 1.4625 ||:  91%|█████████ | 61/67 [00:01<00:00, 48.76it/s]
accuracy: 0.5567, loss: 1.4577 ||: 100%|██████████| 67/67 [00:01<00:00, 53.47it/s]
  0%|          | 0/268 [00:00<?, ?it/s]
accuracy: 

accuracy: 0.5695, loss: 1.4090 ||:  95%|█████████▍| 254/268 [00:11<00:00, 20.27it/s]
accuracy: 0.5703, loss: 1.4073 ||:  96%|█████████▌| 257/268 [00:11<00:00, 20.40it/s]
accuracy: 0.5703, loss: 1.4068 ||:  97%|█████████▋| 260/268 [00:11<00:00, 20.41it/s]
accuracy: 0.5697, loss: 1.4077 ||:  98%|█████████▊| 263/268 [00:11<00:00, 21.01it/s]
accuracy: 0.5698, loss: 1.4082 ||:  99%|█████████▉| 266/268 [00:11<00:00, 21.37it/s]
accuracy: 0.5700, loss: 1.4074 ||: 100%|██████████| 268/268 [00:12<00:00, 22.17it/s]
  0%|          | 0/67 [00:00<?, ?it/s]
accuracy: 0.5625, loss: 1.4617 ||:  12%|█▏        | 8/67 [00:00<00:00, 73.08it/s]
accuracy: 0.5774, loss: 1.4299 ||:  21%|██        | 14/67 [00:00<00:00, 66.83it/s]
accuracy: 0.5850, loss: 1.4112 ||:  30%|██▉       | 20/67 [00:00<00:00, 62.22it/s]
accuracy: 0.5846, loss: 1.4177 ||:  39%|███▉      | 26/67 [00:00<00:00, 58.94it/s]
accuracy: 0.5839, loss: 1.4160 ||:  46%|████▋     | 31/67 [00:00<00:00, 55.55it/s]
accuracy: 0.5838, loss: 1.4037 ||:  5

accuracy: 0.5790, loss: 1.3877 ||:  83%|████████▎ | 223/268 [00:09<00:01, 22.54it/s]
accuracy: 0.5796, loss: 1.3860 ||:  84%|████████▍ | 226/268 [00:09<00:01, 23.78it/s]
accuracy: 0.5797, loss: 1.3850 ||:  85%|████████▌ | 229/268 [00:10<00:01, 23.57it/s]
accuracy: 0.5786, loss: 1.3885 ||:  87%|████████▋ | 232/268 [00:10<00:01, 23.60it/s]
accuracy: 0.5794, loss: 1.3873 ||:  88%|████████▊ | 235/268 [00:10<00:01, 21.47it/s]
accuracy: 0.5790, loss: 1.3890 ||:  89%|████████▉ | 238/268 [00:10<00:01, 19.61it/s]
accuracy: 0.5788, loss: 1.3890 ||:  90%|████████▉ | 241/268 [00:10<00:01, 20.61it/s]
accuracy: 0.5791, loss: 1.3876 ||:  91%|█████████ | 244/268 [00:10<00:01, 22.03it/s]
accuracy: 0.5801, loss: 1.3836 ||:  93%|█████████▎| 248/268 [00:10<00:00, 25.05it/s]
accuracy: 0.5799, loss: 1.3845 ||:  94%|█████████▎| 251/268 [00:11<00:00, 24.13it/s]
accuracy: 0.5803, loss: 1.3836 ||:  95%|█████████▍| 254/268 [00:11<00:00, 24.16it/s]
accuracy: 0.5805, loss: 1.3833 ||:  96%|█████████▌| 257/268 [00:1

...

accuracy: 0.7202, loss: 0.9538 ||:  18%|█▊        | 49/268 [00:02<00:09, 23.60it/s]
accuracy: 0.7187, loss: 0.9620 ||:  19%|█▉        | 52/268 [00:02<00:08, 24.51it/s]
accuracy: 0.7192, loss: 0.9571 ||:  21%|██        | 55/268 [00:02<00:08, 24.15it/s]
accuracy: 0.7198, loss: 0.9585 ||:  22%|██▏       | 58/268 [00:02<00:08, 23.68it/s]
accuracy: 0.7191, loss: 0.9593 ||:  23%|██▎       | 61/268 [00:02<00:08, 23.08it/s]
accuracy: 0.7174, loss: 0.9612 ||:  24%|██▍       | 64/268 [00:02<00:08, 23.02it/s]
accuracy: 0.7159, loss: 0.9666 ||:  25%|██▌       | 67/268 [00:02<00:08, 23.56it/s]
accuracy: 0.7183, loss: 0.9579 ||:  26%|██▌       | 70/268 [00:03<00:08, 23.85it/s]
accuracy: 0.7203, loss: 0.9533 ||:  27%|██▋       | 73/268 [00:03<00:08, 23.88it/s]
accuracy: 0.7202, loss: 0.9526 ||:  28%|██▊       | 76/268 [00:03<00:08, 23.92it/s]
accuracy: 0.7235, loss: 0.9429 ||:  29%|██▉       | 79/268 [00:03<00:07, 24.79it/s]
accuracy: 0.7236, loss: 0.9399 ||:  31%|███       | 82/268 [00:03<00:07, 24.

accuracy: 0.7378, loss: 0.8486 ||:   6%|▌         | 15/268 [00:00<00:12, 19.79it/s]
accuracy: 0.7417, loss: 0.8486 ||:   7%|▋         | 18/268 [00:00<00:12, 20.68it/s]
accuracy: 0.7405, loss: 0.8725 ||:   8%|▊         | 21/268 [00:00<00:11, 21.27it/s]
accuracy: 0.7403, loss: 0.8852 ||:   9%|▉         | 24/268 [00:01<00:11, 22.02it/s]
accuracy: 0.7358, loss: 0.8865 ||:  10%|█         | 27/268 [00:01<00:11, 21.56it/s]
accuracy: 0.7350, loss: 0.8891 ||:  11%|█         | 30/268 [00:01<00:10, 21.90it/s]
accuracy: 0.7338, loss: 0.8938 ||:  12%|█▏        | 33/268 [00:01<00:10, 23.11it/s]
accuracy: 0.7333, loss: 0.8967 ||:  13%|█▎        | 36/268 [00:01<00:10, 23.14it/s]
accuracy: 0.7333, loss: 0.8965 ||:  15%|█▍        | 39/268 [00:01<00:09, 23.32it/s]
accuracy: 0.7345, loss: 0.8898 ||:  16%|█▌        | 42/268 [00:01<00:09, 23.50it/s]
accuracy: 0.7330, loss: 0.8987 ||:  17%|█▋        | 45/268 [00:02<00:09, 23.20it/s]
accuracy: 0.7291, loss: 0.9100 ||:  18%|█▊        | 48/268 [00:02<00:09, 22.

accuracy: 0.6996, loss: 1.0550 ||:  63%|██████▎   | 42/67 [00:00<00:00, 47.92it/s]
accuracy: 0.7011, loss: 1.0575 ||:  70%|███████   | 47/67 [00:00<00:00, 47.03it/s]
accuracy: 0.7016, loss: 1.0532 ||:  78%|███████▊  | 52/67 [00:01<00:00, 47.19it/s]
accuracy: 0.7014, loss: 1.0569 ||:  87%|████████▋ | 58/67 [00:01<00:00, 50.28it/s]
accuracy: 0.7046, loss: 1.0451 ||:  97%|█████████▋| 65/67 [00:01<00:00, 53.23it/s]
accuracy: 0.7030, loss: 1.0468 ||: 100%|██████████| 67/67 [00:01<00:00, 50.16it/s]
  0%|          | 0/268 [00:00<?, ?it/s]
accuracy: 0.7583, loss: 0.8135 ||:   1%|          | 2/268 [00:00<00:14, 18.93it/s]
accuracy: 0.7400, loss: 0.8501 ||:   2%|▏         | 5/268 [00:00<00:13, 19.82it/s]
accuracy: 0.7500, loss: 0.8547 ||:   3%|▎         | 8/268 [00:00<00:12, 21.06it/s]
accuracy: 0.7485, loss: 0.8487 ||:   4%|▍         | 11/268 [00:00<00:11, 21.59it/s]
accuracy: 0.7440, loss: 0.8613 ||:   5%|▌         | 14/268 [00:00<00:11, 22.05it/s]
accuracy: 0.7422, loss: 0.8582 ||:   6%|▋    

accuracy: 0.7296, loss: 0.9069 ||:  98%|█████████▊| 263/268 [00:11<00:00, 23.89it/s]
accuracy: 0.7299, loss: 0.9058 ||:  99%|█████████▉| 266/268 [00:11<00:00, 23.56it/s]
accuracy: 0.7304, loss: 0.9051 ||: 100%|██████████| 268/268 [00:11<00:00, 23.51it/s]
  0%|          | 0/67 [00:00<?, ?it/s]
accuracy: 0.7333, loss: 0.9521 ||:   7%|▋         | 5/67 [00:00<00:01, 46.98it/s]
accuracy: 0.7030, loss: 1.0352 ||:  16%|█▋        | 11/67 [00:00<00:01, 49.47it/s]
accuracy: 0.7245, loss: 0.9720 ||:  25%|██▌       | 17/67 [00:00<00:00, 51.99it/s]
accuracy: 0.7043, loss: 1.0357 ||:  34%|███▍      | 23/67 [00:00<00:00, 53.79it/s]
accuracy: 0.7103, loss: 1.0209 ||:  43%|████▎     | 29/67 [00:00<00:00, 53.26it/s]
accuracy: 0.7088, loss: 1.0182 ||:  54%|█████▎    | 36/67 [00:00<00:00, 55.61it/s]
accuracy: 0.7052, loss: 1.0350 ||:  63%|██████▎   | 42/67 [00:00<00:00, 53.69it/s]
accuracy: 0.7000, loss: 1.0450 ||:  72%|███████▏  | 48/67 [00:00<00:00, 51.96it/s]
accuracy: 0.7016, loss: 1.0408 ||:  79%|███

{'best_epoch': 28,
 'peak_cpu_memory_MB': 780.648,
 'training_duration': '00:07:13',
 'training_start_epoch': 0,
 'training_epochs': 29,
 'epoch': 29,
 'training_accuracy': 0.7303805364940736,
 'training_loss': 0.9050629405833003,
 'training_cpu_memory_MB': 780.648,
 'validation_accuracy': 0.7007462686567164,
 'validation_loss': 1.049820852813436,
 'best_validation_accuracy': 0.7029850746268657,
 'best_validation_loss': 1.0467508195051507}

In [107]:
# Manually test predictions
from allennlp.predictors import Predictor

class OwnPredictor(Predictor):
    # Takes as input model and dataset
    
    # define function for prediction, code similar to tutorial
    def predict_language(self, names):
        tag_logits = self.predict_instance(self._dataset_reader.text_to_instance(names)) 
        # No method to get instances other than using hidden variables
        tag_ids = np.argmax(tag_logits['tag_logits'], axis=-1)
        return [self._model.vocab.get_token_from_index(i, 'labels') for i in tag_ids]

predictor = OwnPredictor(model, dataset_reader=dataset_reader)
print(predictor.predict_language(["Kuznetsov", "Schneider", "Washington", "Lindemann", "Müller"]))

['Russian', 'English', 'English', 'English', 'English']
